In [1]:
from dask.distributed import Client
import dask.dataframe as dd
import dask

import pandas as pd
import numpy as np

# ploting
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
client = Client()
client

/opt/anaconda3/lib/python3.8/site-packages/distributed/node.py:151: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 57720 instead
  warnings.warn(


Client Scheduler: tcp://127.0.0.1:57721 Dashboard: http://127.0.0.1:57720/status,Cluster Workers: 4 Cores: 4 Memory: 8.59 GB


In [8]:
dtypes={
       'row_id': 'int32',
       'timestamp': 'int64', 
       'user_id': 'int32',
       'content_id': 'int16',
       'content_type_id': 'int8',
       'task_container_id': 'int16',
       'user_answer': 'int8', 
       'answered_correctly': 'int8',
       'prior_question_elapsed_time': 'float32', 
       'prior_question_had_explanation': 'bool',
   }

train = dd.read_csv("./data/train.csv", delimiter=',', low_memory=False)
questions = pd.read_csv("./data/questions.csv",)
lectures = pd.read_csv("./data/lectures.csv",)

questions['content_type_id'] = 0


In [11]:
train.user_id.max().compute()

2147482888

In [14]:
train.dtypes

row_id                              int64
timestamp                           int64
user_id                             int64
content_id                          int64
content_type_id                     int64
task_container_id                   int64
user_answer                         int64
answered_correctly                  int64
prior_question_elapsed_time       float64
prior_question_had_explanation     object
dtype: object

### User Level Analysis

instead of analyzing the entire dataset at a time, here i am focusing on a single user data to better understand the how data is generated and the features 

In [18]:
user_interaction_count = train.sample(frac=0.1).groupby("user_id").row_id.count().compute()

In [19]:
train_user = train.loc[train.user_id == 801103753].compute()

In [21]:
train_user.isna().sum(axis=0)

row_id                              0
timestamp                           0
user_id                             0
content_id                          0
content_type_id                     0
task_container_id                   0
user_answer                         0
answered_correctly                  0
prior_question_elapsed_time       309
prior_question_had_explanation      1
dtype: int64

In [22]:
train_user[train_user.prior_question_elapsed_time.isna()]

,row_id,timestamp,user_id,content_id,content_type_id,task_container_id,user_answer,answered_correctly,prior_question_elapsed_time,prior_question_had_explanation
398131,37497771,0,801103753,4886,0,0,1,0,NaN,NaN
398172,37497812,148074210,801103753,3153,1,42,-1,-1,NaN,False
398199,37497839,205559895,801103753,21411,1,69,-1,-1,NaN,False
398219,37497859,385005893,801103753,8976,1,77,-1,-1,NaN,False
398243,37497883,512863699,801103753,5752,1,113,-1,-1,NaN,False
...,...,...,...,...,...,...,...,...,...,...
415425,37515065,27800679566,801103753,4705,1,9536,-1,-1,NaN,False
415592,37515232,27896097838,801103753,19147,1,9636,-1,-1,NaN,False
415656,37515296,27936740093,801103753,24402,1,9684,-1,-1,NaN,False
415911,37515551,28137140378,801103753,15722,1,9881,-1,-1,NaN,False


In [47]:
train_user.groupby(train_user.content_type_id). \
apply(lambda x: x['prior_question_elapsed_time'].isna().sum()). \
to_frame().rename({0:'prior_question_elapsed_time_NaNs_count'}, axis=1)

,prior_question_elapsed_time_NaNs_count
content_type_id,
0,1
1,308


In [89]:
questions['content_type_id'] = 0

master_data = pd.merge(left=train_user[train_user.content_type_id == 0], 
                       right=questions, 
                       left_on=["content_id", "content_type_id"],
                       right_on=["question_id","content_type_id"],
                       how="left",
                      validate="m:1")

assert master_data.shape[0] == train_user[train_user.content_type_id == 0].shape[0]

In [90]:
master_data.isna().sum()

row_id                            0
timestamp                         0
user_id                           0
content_id                        0
content_type_id                   0
task_container_id                 0
user_answer                       0
answered_correctly                0
prior_question_elapsed_time       1
prior_question_had_explanation    1
question_id                       0
bundle_id                         0
correct_answer                    0
part                              0
tags                              0
dtype: int64

### Repeated Questions

In [77]:
df = master_data.groupby(master_data.question_id). \
agg({'row_id': ['count'], 'bundle_id': 'max',})


In [83]:
df.columns = ['_'.join(col).strip() for col in df.columns.values]

In [87]:
df.rename({'row_id_count': 'count', "bundle_id": 'question_bundle_id'}, axis=1).sort_values("count", ascending=False)

,count,bundle_id_max
question_id,,
3348,9,3348
3349,9,3348
3350,9,3348
853,8,853
1756,8,1754
...,...,...
8762,1,8762
8763,1,8763
8764,1,8764


In [91]:
master_data[master_data.question_id == 3348]

,row_id,timestamp,user_id,content_id,content_type_id,task_container_id,user_answer,answered_correctly,prior_question_elapsed_time,prior_question_had_explanation,question_id,bundle_id,correct_answer,part,tags
2447,37500328,3947698087,801103753,3348,0,1648,1,0,30000.0,True,3348,3348,0,4,106 67 81
5010,37502968,6119074538,801103753,3348,0,3077,1,0,30666.0,True,3348,3348,0,4,106 67 81
5038,37502996,6156802197,801103753,3348,0,3090,0,1,28333.0,True,3348,3348,0,4,106 67 81
5129,37503089,6233008928,801103753,3348,0,3149,0,1,10000.0,True,3348,3348,0,4,106 67 81
5167,37503128,6282655073,801103753,3348,0,3171,0,1,21000.0,True,3348,3348,0,4,106 67 81
5169,37503130,6282687298,801103753,3348,0,3178,0,1,4333.0,True,3348,3348,0,4,106 67 81
7712,37505696,9095340064,801103753,3348,0,4738,1,0,24666.0,True,3348,3348,0,4,106 67 81
7798,37505783,9136362894,801103753,3348,0,4783,0,1,24666.0,True,3348,3348,0,4,106 67 81
7943,37505928,9408109001,801103753,3348,0,4840,0,1,23000.0,True,3348,3348,0,4,106 67 81


In [94]:
not_repeated_questions = master_data.sort_values("timestamp").drop_duplicates(['question_id'], keep='first')

In [95]:
not_repeated_questions

,row_id,timestamp,user_id,content_id,content_type_id,task_container_id,user_answer,answered_correctly,prior_question_elapsed_time,prior_question_had_explanation,question_id,bundle_id,correct_answer,part,tags
0,37497771,0,801103753,4886,0,0,1,0,NaN,NaN,4886,4886,3,5,8
1,37497772,21186,801103753,4032,0,1,3,0,249000.0,False,4032,4032,1,5,173
2,37497773,55030,801103753,5927,0,2,1,1,18000.0,False,5927,5927,1,5,89
3,37497774,81839,801103753,4754,0,3,2,1,31000.0,False,4754,4754,2,5,53
4,37497775,103346,801103753,6336,0,4,1,0,24000.0,False,6336,6336,0,5,53
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17603,37515682,28340412348,801103753,9548,0,9994,2,1,44000.0,True,9548,9548,2,5,73
17604,37515683,28340635545,801103753,13507,0,9995,0,1,36000.0,True,13507,13507,0,5,79
17605,37515684,28340706852,801103753,9203,0,9996,2,1,136000.0,True,9203,9203,2,5,96
17606,37515685,28340784904,801103753,9819,0,9997,0,0,35000.0,True,9819,9819,2,5,1


In [101]:
repeated_questions = master_data.loc[~master_data.index.isin(not_repeated_questions.index)]

In [105]:
repeated_questions[repeated_questions.question_id == 1398]

,row_id,timestamp,user_id,content_id,content_type_id,task_container_id,user_answer,answered_correctly,prior_question_elapsed_time,prior_question_had_explanation,question_id,bundle_id,correct_answer,part,tags
67,37497840,295274091,801103753,1398,0,70,0,1,19000.0,True,1398,1398,0,2,62 155 163 38 81
87,37497861,385039564,801103753,1398,0,90,0,1,8000.0,True,1398,1398,0,2,62 155 163 38 81
584,37498395,1235919526,801103753,1398,0,577,0,1,12000.0,True,1398,1398,0,2,62 155 163 38 81
9618,37507619,11384875424,801103753,1398,0,5651,0,1,26000.0,True,1398,1398,0,2,62 155 163 38 81


In [103]:
not_repeated_questions.answered_correctly.mean()

0.6805168764620697

In [104]:
repeated_questions.answered_correctly.mean()

0.7633225208526413

In [113]:
repeated_vs_non_repeated = train.loc[1, ['row_id', 'timestamp',]].compute()

KeyboardInterrupt: 

In [126]:
train = dd.read_csv("./data/train.csv",
                    usecols=['row_id', 'user_id', 'timestamp', 
                             'content_id', 'content_type_id', 
                             'task_container_id', 'answered_correctly', ],)

In [7]:
train[train.content_type_id == 0].merge(questions[['question_id', 'bundle_id', 'content_type_id']], left_on=['content_id', 'content_type_id'],
           right_on = ['question_id', 'content_type_id'], how='left',). \
           groupby(['user_id', 'question_id']).agg({'row_id': ['count'], 'bundle_id': 'max',}). \
           rename(columns={'row_id_count': 'count', "bundle_id": 'question_bundle_id'},). \
            to_csv("./tmp/question_count_*.csv")

distributed.nanny - WARNING - Worker exceeded 95% memory budget. Restarting
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Worker exceeded 95% memory budget. Restarting
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Worker exceeded 95% memory budget. Restarting
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Worker exceeded 95% memory budget. Restarting
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Worker exceeded 95% memory budget. Restarting
distributed.nanny - WARNING - Restarting worker


KilledWorker: ("('aggregate-combine-23c1edfaf941862812ad78f97ad4f1fa', 2, 0, 0)", <Worker 'tcp://127.0.0.1:57727', name: 0, memory: 0, processing: 1>)

In [5]:
import os
os.mkdir("tmp")